In [27]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Initialize webcam
cap = cv2.VideoCapture(0)
cam_width, cam_height = 640, 480  # Set webcam resolution
cap.set(3, cam_width)
cap.set(4, cam_height)

screen_width, screen_height = pyautogui.size()

dragging = False
smooth_factor = 5  # Adjust this to control cursor smoothing
prev_x, prev_y = 0, 0  # Store previous cursor positions

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    # Convert the image color from BGR to RGB, and process it with MediaPipe Hands
    image = cv2.flip(image, 1)  # Flip for a mirror effect
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]

            # Map coordinates from webcam space to screen space
            x = int(np.interp(index_tip.x * cam_width, [0, cam_width], [0, screen_width]))
            y = int(np.interp(index_tip.y * cam_height, [0, cam_height], [0, screen_height]))

            # Smooth cursor movement
            curr_x = (prev_x + x) // smooth_factor
            curr_y = (prev_y + y) // smooth_factor
            pyautogui.moveTo(curr_x, curr_y)
            prev_x, prev_y = curr_x, curr_y

            # Detect gestures
            index_thumb_distance = ((index_tip.x - thumb_tip.x) ** 2 + (index_tip.y - thumb_tip.y) ** 2) ** 0.5

            # Left Click
            if index_thumb_distance < 0.05 and not dragging:
                pyautogui.click()
                dragging = True
            elif index_thumb_distance > 0.1 and dragging:
                dragging = False

    cv2.imshow('Virtual Mouse', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

AttributeError: module 'matplotlib' has no attribute 'rcParams'

In [26]:
pip install opencv-python mediapipe pyautogui streamlit


5054.58s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install numpy

5032.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


In [31]:
import cv2
import mediapipe as mp
import time

# Initialize Mediapipe Hand Detection
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

# Initialize Video Capture
cap = cv2.VideoCapture(0)
prev_time = 0  # FPS Calculation

# Finger names mapped to their landmark index
FINGER_NAMES = {4: "Thumb", 8: "Index", 12: "Middle", 16: "Ring", 20: "Pinky"}

# Function to determine left or right hand (Corrected for flipped video)
def get_hand_label(hand_landmarks, image_width):
    """ Determine if the hand is left or right (considering flipped video). """
    thumb_tip_x = hand_landmarks.landmark[4].x * image_width
    index_finger_tip_x = hand_landmarks.landmark[8].x * image_width

    if thumb_tip_x > index_finger_tip_x:  # Normal condition
        return "Left Hand", (0, 255, 0)  # Green for left hand
    else:
        return "Right Hand", (255, 0, 0)  # Blue for right hand

# Function to draw a bounding box around the hand
def draw_hand_box(image, hand_landmarks):
    x_min, y_min = float('inf'), float('inf')
    x_max, y_max = 0, 0

    for landmark in hand_landmarks.landmark:
        x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
        x_min, y_min = min(x_min, x), min(y_min, y)
        x_max, y_max = max(x_max, x), max(y_max, y)

    cv2.rectangle(image, (x_min - 20, y_min - 20), (x_max + 20, y_max + 20), (0, 255, 255), 3)
    return x_min, y_min

# Function to label fingers
def label_fingers(image, hand_landmarks):
    for idx, name in FINGER_NAMES.items():
        landmark = hand_landmarks.landmark[idx]
        x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
        cv2.putText(image, name, (x - 30, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

# Main Loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Mirror the frame
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    # Calculate FPS
    curr_time = time.time()
    fps = int(1 / (curr_time - prev_time))
    prev_time = curr_time

    # Get camera resolution
    height, width, _ = frame.shape

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Determine left or right hand (Corrected)
            hand_label, color = get_hand_label(hand_landmarks, width)

            # Draw hand landmarks
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Draw Bounding Box
            x_min, y_min = draw_hand_box(frame, hand_landmarks)

            # Label fingers
            label_fingers(frame, hand_landmarks)

            # Display hand information
            cv2.putText(frame, hand_label, (x_min, y_min - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)

    # Display FPS and tracking status
    cv2.putText(frame, f"FPS: {fps}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 3)
    cv2.putText(frame, f"Resolution: {width}x{height}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (200, 200, 200), 2)
    cv2.putText(frame, f"Tracking Status: {'Hands Detected' if results.multi_hand_landmarks else 'No Hands'}",
                (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Show the frame
    cv2.imshow("Hand Tracking", frame)

    # Exit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release Resources
cap.release()
cv2.destroyAllWindows()

AttributeError: module 'matplotlib' has no attribute 'rcParams'

In [30]:
pip install --upgrade matplotlib numpy opencv-python mediapipe  # Exit the virtual environment


5262.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached numpy-2.2.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
Note: you may need to restart the kernel to use updated packages.
